<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1ceb2e01c7f5ec57ed2bfd885ccd2ca68936192335e90675f5a9ef14552ae463
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 16:09:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 38.22 K (0.27%)
Current PnL: -28.26 L (-18.26%)
CY Booked + Current PnL: -13.97 L (-9.03%)
-------------------
Total profit:  1.42 L
Total loss:  -29.68 L
-------------------
Total Booked + Current PnL: 13.01 L (10.18%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 92.89 L (66.35%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.09,-16.73,20.21,0.09,16959.0,-16865.0,83912.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.24,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.15,-12.35,14.11,0.02,21589.0,-21555.0,153004.0,133.26,53.0,M-SC,15.83,234.0,-1.00,1.13,33.09,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.97,-1.09,23.39,22.05,22668.0,-1065.0,96915.0,-14.63,58.0,H-SC,6.97,164.0,-0.05,0.72,40.87,XR,ATH,FINANCE
82,VOLTAS,1530.00,1.76,7.74,11.09,19.69,22910.0,14838.0,206580.0,-2.74,55.0,X-MC,2.60,78.0,0.65,1.53,15.06,XY25,NTT,AC
43,ITC,452.00,-0.69,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,42.0,X-LC,1.85,5.0,-0.18,1.45,3.20,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,6.16,-14.57,72.02,46.95,77088.0,-18260.0,107037.0,-21.09,43.0,H-SC,6.76,154.0,-0.24,0.79,34.09,MH,ATH,POWER
22,DIXON,18931.72,3.01,-14.08,41.50,21.58,49971.0,-19728.0,120411.0,-56.66,37.0,X-MC,7.71,56.0,-0.39,0.89,8.32,X40N,ATH,IT
10,BANDHANBNK,400.00,2.95,-28.05,167.43,92.42,335570.0,-78135.0,200424.0,40.80,50.0,H-SC,8.72,170.0,-0.23,1.48,15.83,XY24,NTT,BANKS
46,JSWINFRA,342.00,2.72,-7.60,24.70,15.23,45724.0,-15228.0,185119.0,-21.02,49.0,X-MC,8.84,66.0,-0.33,1.37,22.20,X40N,NTT,REALTY
67,SONACOMS,806.63,1.94,-14.80,63.78,39.54,54971.0,-14971.0,86188.0,-33.43,52.0,M-SC,4.63,220.0,-0.27,0.64,21.53,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-2.51,-44.91,145.55,35.27,91047.0,-50995.0,62554.0,-57.75,32.0,L-SC,11.07,270.0,-0.56,0.46,53.10,XR,NTT,HOTELS
57,RAJOOENG,143.10,-2.16,-38.08,125.46,39.61,79629.0,-39030.0,63470.0,-58.14,24.0,H-SC,28.08,135.0,-0.49,0.47,0.00,AR,ATH,MISC
33,HINDUNILVR,2922.00,-1.95,-12.06,29.26,13.67,66145.0,-31003.0,226060.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.67,6.72,XY25,NTT,FMCG
63,SFL,1287.00,-1.79,-43.17,122.84,26.63,183041.0,-113210.0,149008.0,12.80,28.0,M-SC,11.11,239.0,-0.62,1.10,0.00,XY24,NTT,MISC
26,GILLETTE,11206.78,-1.70,-3.81,37.67,32.42,91996.0,-9681.0,244215.0,-20.93,39.0,X-SC,9.15,85.0,-0.11,1.81,10.71,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.22,-6.26,121.79,107.91,168567.0,-9236.0,138408.0,-23.79,41.0,M-SC,12.49,216.0,-0.05,1.02,3.20,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.15,-12.35,14.11,0.02,21589.0,-21555.0,153004.0,133.26,53.0,M-SC,15.83,234.0,-1.00,1.13,33.09,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.09,-16.73,20.21,0.09,16959.0,-16865.0,83912.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.24,OX40N,NTT,DURABLES
66,SIS,528.00,0.14,-25.39,62.54,21.26,51806.0,-28195.0,82837.0,1952.85,44.0,H-SC,4.78,166.0,-0.54,0.61,11.92,OX40N,NTT,MISC
47,KANSAINER,340.00,0.11,-25.41,52.13,13.47,104859.0,-68517.0,201150.0,-69.28,31.0,H-SC,4.70,158.0,-0.65,1.49,2.28,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.52,6.89,61.17,72.27,110789.0,11669.0,181117.0,-8.35,68.0,M-LC,2.02,99.0,0.11,1.34,13.49,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.52,6.89,61.17,72.27,110789.0,11669.0,181117.0,-8.35,68.0,M-LC,2.02,99.0,0.11,1.34,13.49,XR,NTT,IT
38,INDIAMART,4810.62,-0.30,-4.08,115.52,106.72,136661.0,-5035.0,118301.0,-52.54,31.0,H-SC,2.31,139.0,-0.04,0.88,18.34,AR,ATH,MISC
37,IEX,219.00,0.53,-2.82,52.93,48.63,102855.0,-5632.0,194322.0,-35.29,52.0,H-SC,14.02,136.0,-0.05,1.44,8.41,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.72,-2.02,38.42,35.63,78892.0,-4232.0,205342.0,-15.85,43.0,H-MC,3.10,119.0,-0.05,1.52,14.32,AR,ATH,PHARMA
50,MASFIN,398.61,0.97,-1.09,23.39,22.05,22668.0,-1065.0,96915.0,-14.63,58.0,H-SC,6.97,164.0,-0.05,0.72,40.87,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.20,-21.78,89.79,48.45,140820.0,-43673.0,156833.0,-27.39,24.0,X-MC,12.70,64.0,-0.31,1.16,2.11,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-1.95,-12.06,29.26,13.67,66145.0,-31003.0,226060.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.67,6.72,XY25,NTT,FMCG
75,TMPV,600.00,0.23,-17.67,72.69,42.18,165428.0,-48830.0,227580.0,-26.74,31.0,X-LC,6.37,3.0,-0.30,1.68,1.18,XY24,NTT,AUTO
53,PAGEIND,51605.08,-0.36,-9.78,39.74,26.07,58704.0,-16020.0,147720.0,-33.51,32.0,X-MC,12.41,55.0,-0.27,1.09,0.64,X40,ATH,APPARELS
3,ACC,3906.00,-0.40,-25.54,120.47,64.15,213437.0,-60781.0,177170.0,-56.31,33.0,X-SC,2.29,82.0,-0.28,1.31,0.00,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-0.20,-80.21,530.56,24.79,264447.0,-202013.0,49843.0,-81.09,44.0,X-SC,0.93,86.0,-0.76,0.37,21.85,X40N,NTT,MISC
43,ITC,452.0,-0.69,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,42.0,X-LC,1.85,5.0,-0.18,1.45,3.20,X40,NTT,FMCG
8,AWL,485.0,0.69,-21.16,94.70,53.50,225282.0,-63861.0,237890.0,-61.84,36.0,X-MC,2.08,58.0,-0.28,1.76,5.66,XY24,NTT,FMCG
36,ICICIPRULI,790.0,1.84,7.35,22.00,30.97,41741.0,12994.0,189732.0,-17.94,64.0,X-MC,2.17,75.0,0.31,1.40,20.84,X40,ATH,INSURANCE
3,ACC,3906.0,-0.40,-25.54,120.47,64.15,213437.0,-60781.0,177170.0,-56.31,33.0,X-SC,2.29,82.0,-0.28,1.31,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.40,-25.54,120.47,64.15,213437.0,-60781.0,177170.0,-56.31,33.0,X-SC,2.29,82.0,-0.28,1.31,0.00,XY24,BTT,CEMENT
53,PAGEIND,51605.08,-0.36,-9.78,39.74,26.07,58704.0,-16020.0,147720.0,-33.51,32.0,X-MC,12.41,55.0,-0.27,1.09,0.64,X40,ATH,APPARELS
75,TMPV,600.00,0.23,-17.67,72.69,42.18,165428.0,-48830.0,227580.0,-26.74,31.0,X-LC,6.37,3.0,-0.30,1.68,1.18,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.03,-38.06,118.28,35.20,94270.0,-48969.0,79701.0,4.55,34.0,X-SC,15.89,92.0,-0.52,0.59,1.24,X40,NTT,FOOTWEAR
31,HAVELLS,2069.16,0.51,-9.75,46.79,32.48,138506.0,-31975.0,296016.0,-14.34,41.0,X-MC,5.68,67.0,-0.23,2.19,1.42,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.62,-29.42,102.38,42.84,46975.0,-19123.0,45883.0,-53.72,43.0,X-SC,36.18,83.0,-0.41,0.34,3.81,XY24,NTT,MISC
15,CAMS,4762.00,-0.20,-80.21,530.56,24.79,264447.0,-202013.0,49843.0,-81.09,44.0,X-SC,0.93,86.0,-0.76,0.37,21.85,X40N,NTT,MISC
59,RELAXO,1176.00,-0.41,-47.54,190.33,52.31,144934.0,-69011.0,76149.0,-44.15,42.0,X-SC,6.66,91.0,-0.48,0.56,1.93,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.03,-38.06,118.28,35.20,94270.0,-48969.0,79701.0,4.55,34.0,X-SC,15.89,92.0,-0.52,0.59,1.24,X40,NTT,FOOTWEAR
22,DIXON,18931.72,3.01,-14.08,41.50,21.58,49971.0,-19728.0,120411.0,-56.66,37.0,X-MC,7.71,56.0,-0.39,0.89,8.32,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,0.90,-12.50,36.31,19.28,109920.0,-43229.0,302727.0,-23.62,64.0,X-LC,4.93,1.0,-0.39,2.24,11.91,X40,ATH,IT
41,INFY,2275.00,0.01,8.80,42.35,54.87,144167.0,27529.0,340417.0,-15.29,62.0,X-LC,3.32,2.0,0.19,2.52,17.81,X40,BTT,IT
75,TMPV,600.00,0.23,-17.67,72.69,42.18,165428.0,-48830.0,227580.0,-26.74,31.0,X-LC,6.37,3.0,-0.30,1.68,1.18,XY24,NTT,AUTO
81,VBL,671.64,0.30,-3.05,39.94,35.67,122299.0,-9634.0,306208.0,-14.52,58.0,X-LC,4.62,4.0,-0.08,2.27,10.38,X40N,ATH,FMCG
43,ITC,452.00,-0.69,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,42.0,X-LC,1.85,5.0,-0.18,1.45,3.20,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.27,-37.53,112.73,32.89,54371.0,-28976.0,48231.0,-695.74,55.0,L-MC,5.87,259.0,-0.53,0.36,32.82,XR,NTT,BANKS
6,ASIANTILES,137.00,0.19,-15.74,115.61,81.67,91383.0,-14766.0,79044.0,7105.56,48.0,L-SC,13.57,271.0,-0.16,0.59,53.07,XR,NTT,CERAMICS
50,MASFIN,398.61,0.97,-1.09,23.39,22.05,22668.0,-1065.0,96915.0,-14.63,58.0,H-SC,6.97,164.0,-0.05,0.72,40.87,XR,ATH,FINANCE
13,BSOFT,831.70,0.41,-22.10,93.55,50.79,102506.0,-31079.0,109574.0,0.12,67.0,H-SC,5.96,151.0,-0.30,0.81,27.58,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.15,-12.35,14.11,0.02,21589.0,-21555.0,153004.0,133.26,53.0,M-SC,15.83,234.0,-1.00,1.13,33.09,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,0.41,-22.10,93.55,50.79,102506.0,-31079.0,109574.0,0.12,67.0,H-SC,5.96,151.0,-0.30,0.81,27.58,XR,ATH,IT
1,ABB,7934.0,0.68,-3.16,50.32,45.57,127483.0,-8275.0,253344.0,-37.78,60.0,H-MC,4.78,120.0,-0.06,1.88,7.80,AR,NTT,ELECTRICAL
56,RAJESHEXPO,518.0,-0.09,-62.86,169.44,0.07,86323.0,-86231.0,50946.0,1767.42,58.0,L-SC,2.43,268.0,-1.00,0.38,26.71,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,0.52,6.89,61.17,72.27,110789.0,11669.0,181117.0,-8.35,68.0,M-LC,2.02,99.0,0.11,1.34,13.49,XR,NTT,IT
0,5PAISA,593.0,0.15,-41.37,92.56,12.90,118576.0,-90389.0,128107.0,88.70,43.0,H-SC,11.84,161.0,-0.76,0.95,6.52,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.82
1,25,45.72
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.35
LC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.29
X40      22.36
X40N     10.88
XR       10.14
XY25      9.40
AR        9.35
OX40N     7.75
SR        1.02
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.64
X-MC    23.56
X-LC    21.06
M-SC    12.07
X-SC     6.88
H-MC     5.03
L-SC     1.43
M-MC     1.40
M-LC     1.34
H-LC     1.16
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.33,-7.57,42.35
IT,12.80,-16.53,79.53
FINANCE,9.62,-19.47,69.95
ELECTRICAL,6.23,-11.45,52.27
MISC,6.07,-58.44,123.35
PAINTS,6.04,-13.06,29.86
INSURANCE,4.60,-1.52,37.30
PHARMA,4.10,-0.95,33.25
AUTO,3.43,-23.87,76.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3214314.0,21
AR,1310096.0,10
XR,1294876.0,13
X40,1016391.0,14
X40N,977335.0,9
OX40N,744336.0,10
XY25,371497.0,6
SR,283566.0,2
MH,77088.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3653836.0,25
M-SC,1426653.0,15
X-MC,1298149.0,16
X-SC,989271.0,8
X-LC,917636.0,11
H-MC,400266.0,3
L-SC,268753.0,3
M-LC,110789.0,1
H-LC,76351.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1272699.0      6
           AR         901374.0      5
M-SC       XY24       814467.0      6
H-SC       XR         773271.0      7
X-SC       X40N       507229.0      3
X-MC       X40        450410.0      7
           XY24       397874.0      3
X-LC       X40        379715.0      5
H-SC       OX40N      345838.0      4
M-SC       OX40N      312175.0      5
X-SC       XY24       295776.0      3
H-SC       SR         283566.0      2
X-MC       X40N       270390.0      4
X-LC       XY24       239607.0      2
H-MC       AR         206375.0      2
X-LC       X40N       199716.0      2
H-MC       XY24       193891.0      1
X-SC       X40        186266.0      2
L-SC       XR         182430.0      2
X-MC       XY25       179475.0      2
M-SC       XR         174015.0      2
           AR         125996.0      2
M-LC       XR         110789.0      1
X-LC       XY25        98598.0      2
L-SC       OX40N       86323.0      1
H-SC       MH          77088.0      1
H-LC       AR          76351.0      1
M-MC       XY25        55879.0      1
L-MC       XR          54371.0      1
L-LC       XY25        37545.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
